In [8]:
dataset = pd.read_csv('./workdir/annotation_small.csv')
dataset[dataset['Катя (short list)'].isin({1.0, 0.0})].to_csv('./workdir/annotation_small_selected.csv')

In [21]:
global_model_path = './workdir/models/model_word2vec_test1'

In [22]:
import pandas as pd
from argparse import ArgumentParser
from pymystem3 import Mystem
from tqdm import tqdm
from gensim.models import FastText


morph = Mystem()

model_path = global_model_path
vectors_path = './workdir/vectors_parts.txt'
#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'

comp = pd.read_csv(compounds_path)

chast1 = list(comp['Часть 1'].values)
chast2 = list(comp['Часть 2'].values)

print('loading model')
model = FastText.load(model_path)
print('loaded')
print(len(model.wv.vocab))

part1_lem = []
part2_lem = []

print('Starting lemmatize')

for w1, w2 in zip(chast1, chast2):
    lem_w1 = morph.lemmatize(w1)[0]
    lem_w2 = morph.lemmatize(w2)[0]
    part1_lem.append(lem_w1)
    part2_lem.append(lem_w2)

with open(vectors_path, 'w') as vectors:
    for w1, w2 in zip(part1_lem, part2_lem):
        if w1 in model.wv.vocab:
            vectors.write(w1 + ' ' + ' '.join(model.wv[w1].astype('str')) + '\r\n')
        else:
            print(w1)
        if w2 in model.wv.vocab:
            vectors.write(w2 + ' ' + ' '.join(model.wv[w2].astype('str')) + '\r\n')
        else:
            print(w2)

loading model
loaded
2720494
Starting lemmatize


In [27]:
import pandas as pd
from argparse import ArgumentParser
from pymystem3 import Mystem
from tqdm import tqdm
from gensim.models import FastText

morph = Mystem()

#model_path = './workdir/models/model_fast2vec_300_5'
model_path = global_model_path
vector_path = './workdir/vectors_compounds.txt'
#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'


comp = pd.read_csv(compounds_path)

chast1 = list(comp['Часть 1'].values)
chast2 = list(comp['Часть 2'].values)

print('loading model')
model = FastText.load(model_path)
print('loaded')
print(len(model.wv.vocab))

comp_lem = []

print('Starting lemmatize')

for w1, w2 in zip(chast1, chast2):
    lem_w1 = morph.lemmatize(w1)[0]
    lem_w2 = morph.lemmatize(w2)[0]
    comp_lem.append(lem_w1 + '_' + lem_w2)

with open(vector_path, 'w') as vectors:
    for w1 in comp_lem:
        if w1 in model.wv.vocab:
            vectors.write(w1 + ' ' + ' '.join(model.wv[w1].astype('str')) + '\r\n')
        else:
            print(w1)

loading model
loaded
2720494
Starting lemmatize
приговор_судно
красный_черт


In [29]:
len(comp_lem)

201

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.spatial import distance
from pymystem3 import Mystem
from argparse import ArgumentParser


#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'

morph = Mystem()

def acquiring(wordvecs, compvecs):
    comp = pd.read_csv(compounds_path)

    part1 = list(comp['Часть 1'].values)
    part2 = list(comp['Часть 2'].values)
    values = list(comp['Катя (short list)'].values)
    
    print()

    compounds = []
    classes = []

    for w1, w2, v in zip(part1, part2, values):
        if v != 2:
            lem_w1 = morph.lemmatize(w1)[0]
            lem_w2 = morph.lemmatize(w2)[0]
            compounds.append('_'.join([lem_w1, lem_w2]))
            classes.append(v)

    vecs1 = []
    vecs2 = []
    vecsc = []
    true_comp_class = []

    words = []
    comps = []
    
    with open(wordvecs) as w:
        for line in w:
            words.append(line.split())

    with open(compvecs) as c:
        for line in c:
            comps.append(line.split())

    print(len(compounds))
    print(len(values))
    for compound, value in zip(compounds, values):
        comp_flag = 0
        w1_flag = 0
        w2_flag = 0
        for line in comps:
            if compound == line[0]:
                vecc = np.array(line[1:]).astype(np.float32)
                comp_flag = 1
        w1 = compound.split('_')[0]
        w2 = compound.split('_')[1]
        if comp_flag:
            for line in words:
                if w1 == line[0]:
                    vec1 = np.array(line[1:]).astype(np.float32)
                    w1_flag = 1
                    break
            for line in words:
                if w2 == line[0]:
                    w2_flag = 1
                    vec2 = np.array(line[1:]).astype(np.float32)
                    break
        if comp_flag and w1_flag and w2_flag:
            vecs1.append(vec1)
            vecs2.append(vec2)
            vecsc.append(vecc)
            true_comp_class.append(value)
        else:
            print(w1, w2, compound, comp_flag, w1_flag, w2_flag)

    print(len(vecsc), 'examples retrieved for experiment')
    return vecs1, vecs2, vecsc, true_comp_class


def get_mean(part1_vecs, part2_vecs):
    parts_mean = []
    for vec1, vec2 in zip(part1_vecs, part2_vecs):
        parts_mean.append((vec1 + vec2) / 2)

    return parts_mean


def cosine_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean = get_mean(part1_vecs, part2_vecs)

    cosines = []
    for w, c in zip(parts_mean, comp_vecs):
        cosines.append(abs(1 - distance.cosine(w, c)))
    
    print(spearmanr(cosines, true_class)[0])


def chebyshev_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean = get_mean(part1_vecs, part2_vecs)

    chebyshevs = []
    for w, c in zip(parts_mean, comp_vecs):
        chebyshevs.append(abs(distance.chebyshev(w, c)))

    print(spearmanr(chebyshevs, true_class)[0])


def manhattan_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean= get_mean(part1_vecs, part2_vecs)

    manhattans = []
    for w, c in zip(parts_mean, comp_vecs):
        manhattans.append(abs(distance.cityblock(w, c)))
    print(spearmanr(manhattans, true_class)[0])


def euclidean_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean= get_mean(part1_vecs, part2_vecs)

    euclideans = []
    for w, c in zip(parts_mean, comp_vecs):
        euclideans.append(abs(distance.euclidean(w, c)))
    print(spearmanr(euclideans, true_class)[0])


vectors_parts_path = './workdir/vectors_parts.txt'
vectors_compounds_path = './workdir/vectors_compounds.txt'

w1, w2, c, true = acquiring(vectors_parts_path, vectors_compounds_path)

cosine_between_parts_and_compound(w1, w2, c, true)
chebyshev_between_parts_and_compound(w1, w2, c, true)
manhattan_between_parts_and_compound(w1, w2, c, true)
euclidean_between_parts_and_compound(w1, w2, c, true)


201
201
приговор судно приговор_судно 0 0 0
красный черт красный_черт 0 0 0
199 examples retrieved for experiment
0.3593346066740859
-0.07977275893692432
-0.13168458415557954
-0.13239896707143256


In [34]:
model['черт']

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.19120477, -1.4227085 ,  0.07280233, -2.2622147 , -0.05061609,
        0.05105349, -2.208469  , -0.06491596, -0.0322176 ,  1.1179839 ,
        1.22782   ,  2.630733  , -3.0086253 , -0.9810795 , -1.6233798 ,
        1.2038122 ,  0.92346174, -1.2351778 , -0.894103  , -0.976527  ,
        0.01442312,  0.13855833, -0.23599832, -0.07819845, -0.36940575,
        0.95845777, -1.2605244 , -0.87924594, -1.4497571 , -2.4237576 ,
        2.5970576 , -0.0301838 , -0.5671605 ,  0.8208104 , -0.03633777,
        2.1496432 ,  0.5412136 , -1.2187197 , -0.13529265,  0.72096914,
        1.0065191 , -0.23205145,  0.16050196, -1.2166207 , -2.0707355 ,
       -1.2792473 , -1.1825869 ,  0.7249201 , -1.6045942 ,  0.18434829,
        0.42369962,  0.8223133 , -0.9105033 ,  0.13977809,  2.4988577 ,
       -0.28781703,  1.1641194 ,  0.37301946, -1.0414828 , -0.56199586,
       -2.5995567 , -0.7153463 ,  0.20854273, -0.6626351 ,  1.3331832 ,
        0.18200372, -0.58380735, -1.0062047 ,  0.22278368,  1.33